# 函数准备

In [1]:
import os
import numpy as np
import pandas as pd
import json
from copy import deepcopy
import re
import random
from random import choice
random.seed(10)
import shutil
# -*- coding:utf-8 -*
import eval
from mkdir import mk_dir
from bert_format import bset,test_qa

# 是否致病

## Bert 预测

In [39]:
col_ab =  "pat"
standard_0 = pd.DataFrame(np.zeros((0,1)), columns = ["golden"])
prediction_0 = pd.DataFrame(np.zeros((0,3)), columns = ["prediction","rate","cut"])
for re in range(1,11):
    eval_a = json.load(open("eval/evl_%s/input_%s/eval_set.json" % (col_ab, re), "r"))
    standard = pd.DataFrame(eval_a, index=["golden"]).T
    standard_0 = pd.concat((standard_0, standard), axis=0)
    prediction = json.load(open("eval/evl_%s/output_%s/predictions_.json" % (col_ab, re),'r'))
    prediction = pd.DataFrame(prediction, index=["prediction", "rate"]).T
    prediction["cut"] = 1
    for idx in prediction.index:
        prediction.loc[idx, "rate"] = prediction.loc[idx, "rate"][0]
        prediction.loc[idx, "cut"] = 1 - prediction.loc[idx, "rate"]
        if prediction.loc[idx, "rate"] < 0.85:
            prediction.loc[idx, "prediction"] = "no"
    prediction_0 = pd.concat((prediction_0, prediction), axis=0)

fig_path = ("eval/evl_%s/output_%s" % (col_ab, re))
eval.evaluate(standard_0, prediction_0)

结果: 0.8784153005464481 0.7283333333333334 0.8077634011090573 0.7659947414548642


(0.8784153005464481,
 0.7283333333333334,
 0.8077634011090573,
 0.7659947414548642)

## 关键词

In [18]:
pneumonia = {"bronchitis","bronchopneumonia","pneumonia","pneumonitis","pneumonic","pleuropneumonia","pleural"}
LungRelated = {"bronchus","bronchial","tracheobronchial","bronchiolar",      "bronchopulmonary",
               "bronchoalveolar","bronchoscopy",
               "lung","pulmonary","pneumonic","bronchopulmonary","respiratory","respiration"}
infection = {"infection","infections","abscess","pathology","etiology",  "diseases",
             "bacteremia","sepsis","septic","pathogenicity","legionellosis","empyema","necrosis"}

In [21]:
df = pd.read_excel("input/human_label.xlsx")
df['golden'] = 'no'
df.loc[df["是否是致病菌"]=='yes', "golden"] = 'yes'
df["full"] = df["Full"].str.lower()
df["prediction"] = 'no'
for aword in pneumonia:
    df.loc[df["full"].str.contains(aword), "prediction"] = 'yes'
    # print(aword, df.loc[df["full"].str.contains(aword), ['golden','prediction', 'Full','原文链接']])
for aword in LungRelated:
    for bword in infection:
        df.loc[(df["full"].str.contains(aword)) & (df["full"].str.contains(bword)) , "prediction"] = 'yes'
        # print(aword, df.loc[df["full"].str.contains(aword), ['golden','prediction', 'Full','原文链接']])
eval.evaluate(df[['golden']], df[['prediction']])

结果: 0.5305100182149363 0.33774834437086093 0.9426987060998152 0.4973183812774256


(0.5305100182149363,
 0.33774834437086093,
 0.9426987060998152,
 0.4973183812774256)

# 免疫状态

## 关键词

In [22]:
df = pd.read_excel("input/human_label.xlsx")
df = df.loc[df["是否是致病菌"]=='yes', :]
df['golden'] = 'no'
df.loc[df["免疫状态"]=='抑制', "golden"] = 'yes'
df["full"] = df["Full"].str.lower()
df["prediction"] = 'no'
for aword in [
    # "immunodeficiency", 
    "immunocompromised", "immunosuppressive","immunosuppression", 
    # "compromised", "suppressed", "supperssive", "suppression",
    "hiv","hiv-positive","aids",
    "transplantation", "transplanted",
    "malignancy", "cancer", "carcinoma", 
    "leukemia","leukaemia", "lymphoma", "melanoma", "myeloma",
    "impair", "impaired immune", "fibrosis"]:
    df.loc[df["full"].str.contains(aword), "prediction"] = 'yes'
    # print(aword, df.loc[df["full"].str.contains(aword), ['golden','prediction', 'Full','原文链接']])
for aword in ["immunocompetent"]:
    df.loc[df["full"].str.contains(aword), "prediction"] = 'no'


In [23]:
eval.evaluate(df[['golden']], df[['prediction']])

结果: 0.7966728280961183 0.6497175141242938 0.7055214723926381 0.6764705882352942


(0.7966728280961183,
 0.6497175141242938,
 0.7055214723926381,
 0.6764705882352942)

## Bert预测

In [33]:
col_ab = "immu"
standard_0 = pd.DataFrame(np.zeros((0,1)), columns = ["golden"])
prediction_0 = pd.DataFrame(np.zeros((0,3)), columns = ["prediction","rate","cut"])
for re in range(1,11):
    eval_a = json.load(open("eval/evl_%s/input_%s/eval_set.json" % (col_ab, re), "r"))
    standard = pd.DataFrame(eval_a, index=["golden"]).T
    standard_0 = pd.concat((standard_0, standard), axis=0)
    prediction = json.load(open("eval/evl_%s/output_%s/predictions_.json" % (col_ab, re),'r'))
    prediction = pd.DataFrame(prediction, index=["prediction", "rate"]).T
    prediction["cut"] = 1
    for idx in prediction.index:
        prediction.loc[idx, "rate"] = prediction.loc[idx, "rate"][0]
        prediction.loc[idx, "cut"] = 1 - prediction.loc[idx, "rate"]
        if prediction.loc[idx, "rate"] < 0.5:
            prediction.loc[idx, "prediction"] = "no"
    prediction_0 = pd.concat((prediction_0, prediction), axis=0)

fig_path = ("eval/evl_%s/output_%s" % (col_ab, re))
eval.evaluate(standard_0, prediction_0)

结果: 0.744916820702403 0.5654450261780105 0.6625766871165644 0.6101694915254237


(0.744916820702403, 0.5654450261780105, 0.6625766871165644, 0.6101694915254237)

# 预后

In [25]:
df = pd.read_excel("input/human_label.xlsx")
df = df.loc[df["是否是致病菌"]=='yes', :]
df['golden'] = 'no'
df.loc[df["预后"]=='死亡', "golden"] = 'yes'
df["full"] = df["Full"].str.lower()
df["prediction"] = 'no'
for aword in [
    'death', 'dead']:
    df.loc[df["full"].str.contains(aword), "prediction"] = 'yes'
    # print(aword, df.loc[df["full"].str.contains(aword), ['golden','prediction', 'Full','原文链接']])

In [26]:
eval.evaluate(df[['golden']], df[['prediction']])

结果: 0.8983364140480592 0.5161290322580645 0.2857142857142857 0.367816091954023


(0.8983364140480592, 0.5161290322580645, 0.2857142857142857, 0.367816091954023)